In [1]:
import bookutils.setup
from typing import Tuple, List, Callable, Set, Any
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import random
from Fuzzer import Fuzzer, Runner
from Coverage import Coverage, population_coverage, Location

In [2]:
def urlparse_with_exceptions(url: str) -> bool:
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + 
                         repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    # Do something with the URL
    return True

In [3]:
def mutate(s: str, n_mutations: int = 1) -> str:
    """Return s with a random mutation applied"""
    mutators = [
        delete_random_character,
        insert_random_character,
        flip_random_character
    ]

    for i in range(n_mutations):
        mutator = random.choice(mutators)
        s = mutator(s)
    return s

In [4]:
def delete_random_character(s: str) -> str:
    """Returns s with a random character deleted"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    return s[:pos] + s[pos + 1:]

In [5]:
def insert_random_character(s: str) -> str:
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    return s[:pos] + random_character + s[pos:]

In [6]:
def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    return s[:pos] + new_c + s[pos + 1:]

In [7]:
# Class that runs a function with a given input
class FunctionRunner(Runner):
    def __init__(self, function: Callable) -> None:
        """Initialize.  `function` is a function to be executed"""
        self.function = function

    def run_function(self, inp: str) -> Any:
        return self.function(inp)

    def run(self, inp: str) -> Tuple[Any, str]:
        try:
            result = self.run_function(inp)
            outcome = self.PASS
        except Exception:
            result = None
            outcome = self.FAIL

        return result, outcome

In [8]:
# Extends the above to store the coverage as well
class FunctionCoverageRunner(FunctionRunner):
    def run_function(self, inp: str) -> Any:
        with Coverage() as cov:
            try:
                result = super().run_function(inp)
            except Exception as exc:
                self._coverage = cov.coverage()
                raise exc

        self._coverage = cov.coverage()
        return result

    def coverage(self) -> Set[Location]:
        return self._coverage

In [9]:
class MutationFuzzer(Fuzzer):
    """Base class for mutational fuzzing"""

    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        """Constructor.
        `seed` - a list of (input) strings to mutate.
        `min_mutations` - the minimum number of mutations to apply.
        `max_mutations` - the maximum number of mutations to apply.
        """
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        self.population = self.seed
        self.seed_index = 0

    def mutate(self, inp: str) -> str:
        return mutate(inp)

    def create_candidate(self) -> str:
        """Create a new candidate by mutating a population member"""
        candidate = random.choice(self.population)
        trials = random.randint(self.min_mutations, self.max_mutations)
        for i in range(trials):
            candidate = self.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        if self.seed_index < len(self.seed):
            # Still seeding
            self.inp = self.seed[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()
        return self.inp

### For Greybox Fuzzing, we need to refactor our code in two classes: Runner and Fuzzer
### The implementation is omitted from the live coding session but is provided in the materials

In [10]:
# Runner: runs the program and returns coverage info
runner = FunctionCoverageRunner(urlparse_with_exceptions)
runner.run("https://foo.bar/")
# equivalent to "urlparse_with_exceptions("https://foo.bar/")"

(True, 'PASS')

In [12]:
print(list(runner.coverage())[:20])

[('_splitnetloc', 417), ('_coerce_args', 126), ('_splitnetloc', 414), ('urlsplit', 505), ('_coerce_args', 129), ('_coerce_args', 132), ('_noop', 109), ('urlparse_with_exceptions', 2), ('urlsplit', 474), ('urlsplit', 480), ('urlparse_with_exceptions', 11), ('urlparse', 397), ('urlsplit', 477), ('urlparse', 394), ('urlsplit', 489), ('urlparse', 400), ('urlsplit', 486), ('urlsplit', 492), ('urlparse', 401), ('urlsplit', 495)]


In [15]:
class MutationCoverageFuzzer(MutationFuzzer):
    """Fuzz with mutated inputs based on coverage"""

    def reset(self) -> None:
        super().reset()
        self.coverages_seen: Set[frozenset] = set()
        # Now empty; we fill this with seed in the first fuzz runs
        self.population = []
    
    def run(self, runner):
        # Implement
        result, outcome = super().run(runner)
        new_coverage = frozenset(runner.coverage())
        if new_coverage not in self.coverages_seen and outcome == 'PASS':
            self.coverages_seen.add(new_coverage)
            self.population.append(self.inp)

In [17]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutation_coverage_fuzzer = MutationCoverageFuzzer(seed = [seed_input])
mutation_coverage_fuzzer.runs(runner, 1000)
mutation_coverage_fuzzer.population

['http://www.google.com/search?q=fuzzing',
 'http://www.google.com/searcq=fuzzing',
 'http://www.googlFe.comosee4rcq=fuzZing',
 'http://www.google.com/3earch?q=fFUz#zing',
 'http://www.googhe.com/;earc;h?q=fFU{#zing',
 'http://*www/gooGhe.com/;erc;h?qfFU{zing',
 'http://ww.goog,Fe.#o m/se#e4rcq=nuzZing',
 'http://<wwU5w.googhecmm/;earc;hQ=tf|FU{#zing',
 'http://wwU5w.gooRghecmm/;earc;hQ=tf|FU{zing',
 'http://gwgooghe.com/;eqrc;hB/q=5fFU{#zing',
 'http://gkwgooghe.cm/;eqrc;hB/q=fFU{zing']